In [1]:
!pip install transformers byaldi accelerate qwen_vl_utils pdf2image mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.9/517.9 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.5/301.5 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!sudo apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 2s (118 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 126101 

In [3]:
from byaldi import RAGMultiModalModel
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from pdf2image import convert_from_path
import base64
import requests
from urllib.parse import urlencode
import zipfile
import io
import shutil
import os
from mistralai import Mistral

# Загрузка датасета

In [5]:
listdir = os.listdir()

if 'pdf_files' in listdir:
    shutil.rmtree('pdf_files')

if 'docx_files' in listdir:
    shutil.rmtree('docx_files')

os.mkdir('pdf_files')
os.mkdir('docx_files')

listdir = os.listdir()

for filename in listdir:
    if filename.split('.')[-1] == 'pdf':
        os.rename(filename, 'pdf_files/' + filename)

    elif filename.split('.')[-1] == 'docx':
        os.rename(filename, 'docx_files/' + filename)

In [6]:
import os
import zipfile
from pdf2image import convert_from_path

def convert_pdfs_to_images(pdf_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

    for pdf_file in pdf_files:
        pdf_name = os.path.splitext(pdf_file)[0]  # Имя без .pdf
        pdf_path = os.path.join(pdf_folder, pdf_file)
        images = convert_from_path(pdf_path)

        pdf_output_folder = os.path.join(output_folder, pdf_name)
        os.makedirs(pdf_output_folder, exist_ok=True)

        for i, image in enumerate(images):
            image_filename = f"{pdf_name}_page_{i}.png"
            image_path = os.path.join(pdf_output_folder, image_filename)
            image.save(image_path, "PNG")

    print("Преобразование завершено.")

# Пример использования
convert_pdfs_to_images("/content/pdf_files", "/content/output_images")

Преобразование завершено.


In [7]:
import os
import zipfile

def zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)

# Пример использования:
# Архивируем папку /content/output_images в /content/output_images.zip
zip_folder("/content/output_images", "/content/output_images.zip")

# Создание базы знаний

In [4]:
RAG = RAGMultiModalModel.from_pretrained("vidore/colqwen2-v1.0", index_root='/content/indices')

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [8]:
RAG.index(
    input_path="/content/pdf_files",
    index_name='index1',
    store_collection_with_index=False,
    overwrite=True
)

Indexing file: /content/pdf_files/Каталог_оригинальных_запчастей_LADA.pdf
Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Added page 3 of document 0 to index.
Added page 4 of document 0 to index.
Added page 5 of document 0 to index.
Added page 6 of document 0 to index.
Added page 7 of document 0 to index.
Added page 8 of document 0 to index.
Added page 9 of document 0 to index.
Added page 10 of document 0 to index.
Added page 11 of document 0 to index.
Added page 12 of document 0 to index.
Added page 13 of document 0 to index.
Added page 14 of document 0 to index.
Added page 15 of document 0 to index.
Added page 16 of document 0 to index.
Added page 17 of document 0 to index.
Added page 18 of document 0 to index.
Added page 19 of document 0 to index.
Added page 20 of document 0 to index.
Added page 21 of document 0 to index.
Added page 22 of document 0 to index.
Added page 23 of document 0 to index.
Added page 24 of document 0 to index.
Added page 25 of docume

{0: '/content/pdf_files/Каталог_оригинальных_запчастей_LADA.pdf',
 1: '/content/pdf_files/Техническое_обслуживание_и_ремонт_автомобилей.pdf',
 2: '/content/pdf_files/Руководство_по_эксплуатации_Lada\u202fVesta.pdf'}

In [10]:
results = RAG.search('Перечислите три основные жидкости, которые водитель должен регулярно проверять', k=5)

In [11]:
results

[{'doc_id': 1, 'page_num': 326, 'score': 20.25, 'metadata': {}, 'base64': None},
 {'doc_id': 1, 'page_num': 314, 'score': 19.375, 'metadata': {}, 'base64': None},
 {'doc_id': 2, 'page_num': 183, 'score': 19.25, 'metadata': {}, 'base64': None},
 {'doc_id': 2, 'page_num': 180, 'score': 19.125, 'metadata': {}, 'base64': None},
 {'doc_id': 1, 'page_num': 93, 'score': 19.125, 'metadata': {}, 'base64': None}]

In [9]:
def zip_folder(folder_path, output_zip_path):
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Полный путь к файлу
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

folder_to_zip = '/content/indices/index1'
output_zip_file = 'rag.zip'  # Имя выходного ZIP-файла

zip_folder(folder_to_zip, output_zip_file)
print(f"Папка '{folder_to_zip}' успешно заархивирована в '{output_zip_file}'")

Папка '/content/indices/index1' успешно заархивирована в 'rag.zip'
